<h1>Primo Progetto<h1>

In [ ]:
# Importo le librerie richieste
import pandas as pd
from serpapi import GoogleScholarSearch
import json

In [ ]:
GoogleScholarSearch.SERP_API_KEY = "5ad7613cf550e6717a98fe89a1313fd076c01bf67a9ff629de82129283fd92cf"

#### 1 | Utilizzo della libreria Python SerpAPI per scaricare, per ciascuno dei 7 autori elencati nel file nodes.csv

In [ ]:
# Importo il file nodes.csv
df = pd.read_csv('data/nodes.csv')
display(df)

In [ ]:

updated_df = pd.DataFrame(columns=['author_id', 'cited_by', 'interests'])

# Itero su ogni autore del dataframe
for index, row in df.iterrows():
    author_name = row['name']
        
    params = {
        "engine" : "google_scholar_profiles",
        "hl": "en",
        "mauthors": author_name
    }
    search = GoogleScholarSearch(params)
    results = search.get_dict()["profiles"]
    
    for index in range(len(results)):
        new_row = None
        if results[index]['name'] == author_name:
            # Estraggo le informazioni che mi interessano
            author_id = results[index]['author_id']
            cited_by = results[index]['cited_by']
            interests = results[index]['interests']
            interests = interests[0]['title']
    
            # Aggiungo una nuova riga al dataframe updated_df
            new_row = {'author_id': author_id, 'cited_by': cited_by, 'interests': interests}
            updated_df = pd.concat([updated_df, pd.DataFrame([new_row])], ignore_index=True)

df = pd.merge(df, updated_df, left_index=True, right_index=True)
display(df)



In [ ]:
# Sovrascrivo il file nodes.csv con le nuove informazioni trovate
df.to_csv('data/nodes.csv', index=False)

#### 2 | Per ogni autore utilizzo il suo ID per accedere al relativo profilo GoogleScholar e scaricare l’elenco dei suoi coautori, sempre via SerpAPI

In [8]:
df = pd.read_csv('data/nodes.csv')

# creo una lista vuota dove andro ad inserire i nomi dei co-autori
co_authors = []

# Itero su ogni autore del dataframe
for index, row in df.iterrows():
    author_id = row['author_id']
    
    params = {
        "engine" : "google_scholar_author",
        "hl": "en",
        "author_id": author_id
    }
       
    search = GoogleScholarSearch(params)
    results = search.get_dict()['co_authors']
    print(results)
    first_co_author = results[0]['name']
    co_authors.append(first_co_author)
    
print(co_authors)    



[{'name': 'Kevin Roitero', 'link': 'https://scholar.google.com/citations?user=1xd52jMAAAAJ&hl=en', 'serpapi_link': 'https://serpapi.com/search.json?author_id=1xd52jMAAAAJ&engine=google_scholar_author&hl=en', 'author_id': '1xd52jMAAAAJ', 'affiliations': 'University of Udine', 'email': 'Verified email at uniud.it', 'thumbnail': 'https://scholar.googleusercontent.com/citations?view_op=small_photo&user=1xd52jMAAAAJ&citpid=6'}, {'name': 'Stefano Mizzaro', 'link': 'https://scholar.google.com/citations?user=2wvJC6IAAAAJ&hl=en', 'serpapi_link': 'https://serpapi.com/search.json?author_id=2wvJC6IAAAAJ&engine=google_scholar_author&hl=en', 'author_id': '2wvJC6IAAAAJ', 'affiliations': 'Full professor of Computer Science and Information Technology', 'email': 'Verified email at uniud.it', 'thumbnail': 'https://scholar.googleusercontent.com/citations?view_op=small_photo&user=2wvJC6IAAAAJ&citpid=1'}, {'name': 'Gianluca Demartini', 'link': 'https://scholar.google.com/citations?user=PCAiILsAAAAJ&hl=en', 